In [2]:
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch


In [3]:
raw_train_data = pd.read_csv('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\raw-data\\train.csv')
raw_test_data = pd.read_csv('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\raw-data\\test.csv')
raw_valid_data = pd.read_csv('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\raw-data\\valid.csv')
tfidf_train_data = np.load('C:\\Users\\xu741\Desktop\\IML\\a1\\a3\\tfidf-data\\train-tfidf.npy')
tfidf_test_data = np.load('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\tfidf-data\\test-tfidf.npy')
tfidf_valid_data = np.load('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\tfidf-data\\valid-tfidf.npy')
embedding_train_data = np.load('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\embeddings-data\\train-embeddings.npy')
embedding_test_data = np.load('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\embeddings-data\\test-embeddings.npy')
embedding_valid_data = np.load('C:\\Users\\xu741\\Desktop\\IML\\a1\\a3\\embeddings-data\\valid-embeddings.npy')

Define LogisticRegression 

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [5]:
# Define the SVM model
class SVM(nn.Module):
    def __init__(self):
        super(SVM, self).__init__()
        self.linear = nn.Linear(num_features, 1)

    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
from torch.distributions.multivariate_normal import MultivariateNormal

class GaussianNB(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GaussianNB, self).__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.priors = torch.nn.Parameter(torch.zeros(num_classes))
        self.posteriors = torch.nn.Parameter(torch.zeros(num_classes, num_features))
        self.covariances = torch.nn.Parameter(torch.zeros(num_classes, num_features, num_features))

    def forward(self, x):
        x = x.unsqueeze(1).repeat(1, self.num_classes, 1)
        log_probs = MultivariateNormal(self.posteriors, covariance_matrix=self.covariances).log_prob(x)
        log_probs += torch.log_softmax(self.priors, dim=-1).unsqueeze(0).repeat(x.shape[0], 1)
        return log_probs.argmax(dim=-1)

    def fit(self, x, y):
        self.priors.data = torch.log(torch.histc(y, bins=self.num_classes, min=0, max=self.num_classes))
        for c in range(self.num_classes):
            x_c = x[y == c]
            self.posteriors.data[c] = x_c.mean(dim=0)
            self.covariances.data[c] = torch.diag(x_c.var(dim=0))
